# Let's Prediction next season

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import matplotlib.cm as cm
import math
from math import pi
import itertools
import os
import csv
import glob
import pygal
from sklearn.multioutput import MultiOutputRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
%matplotlib inline

In [3]:
game_data= pd.read_csv("/Users/gowthamramakrishnan/Desktop/Liverpool/epl.csv",encoding='ISO-8859-1')
POSITIONS = np.array(range(1, 21))
RANDOM_STATE = 0

In [4]:
game_data

,Unnamed: 0,h_a,xG,xGA,npxG,npxGA,deep,deep_allowed,scored,missed,...,AF.x,AC.x,AY.x,AR.x,B365H.x,B365D.x,B365A.x,HtrgPerc,AtrgPerc,matchDay
0,1,h,2.234560,0.842407,2.234560,0.842407,11,5,4,1,...,9,2,2,0,1.14,10.00,19.00,0.466667,0.416667,Fri
1,2,a,0.842407,2.234560,0.842407,2.234560,5,11,1,4,...,9,2,2,0,1.14,10.00,19.00,0.466667,0.416667,Fri
2,3,a,3.183770,1.200300,2.422640,1.200300,9,1,5,0,...,13,1,2,0,12.00,6.50,1.22,0.600000,0.642857,Sat
3,4,h,1.200300,3.183770,1.200300,2.422640,1,9,0,5,...,13,1,2,0,12.00,6.50,1.22,0.600000,0.642857,Sat
4,5,h,1.340990,1.598640,1.340990,1.598640,4,6,1,1,...,19,4,1,0,1.95,3.60,3.60,0.230769,0.375000,Sat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,572,a,0.445922,2.028830,0.445922,2.028830,3,9,0,4,...,10,1,2,0,1.80,3.90,4.20,0.647059,0.333333,Sun
572,573,a,0.585469,1.738900,0.585469,1.738900,2,3,0,2,...,9,11,4,0,5.00,4.33,1.60,0.500000,0.285714,Sun
573,574,h,1.738900,0.585469,1.738900,0.585469,3,2,2,0,...,9,11,4,0,5.00,4.33,1.60,0.500000,0.285714,Sun
574,575,a,0.672448,2.412390,0.672448,1.651220,2,8,0,4,...,12,0,1,0,1.36,5.25,8.00,0.466667,0.250000,Mon


In [7]:
game_data['scored'] = game_data['scored'].apply(lambda x: str(x))
game_data.sort_values(by=['date', 'h_a'], inplace=True)
game_data.reset_index(inplace=True, drop=True)

# Get game data
results = game_data.groupby(by=['Referee.x', 'date']).agg({'teamId' : ','.join, 'scored' : ','.join}).reset_index()
results['teamA'] = results['teamId'].apply(lambda x: x.split(',')[0])
results['teamB'] = results['teamId'].apply(lambda x: x.split(',')[1])
results['scoredA'] = results['scored'].apply(lambda x: x.split(',')[0]).astype('uint16')
results['scoredB'] = results['scored'].apply(lambda x: x.split(',')[1]).astype('uint16')

# Add additional columns
results.sort_values(by='date', inplace=True)
results.reset_index(inplace=True, drop=True)
results['teamAPosition'] = 0
results['teamBPosition'] = 0
results['teamARecentScored'] = 0
results['teamBRecentScored'] = 0
results['teamARecentConceded'] = 0
results['teamBRecentConceded'] = 0
results['teamARecentPoints'] = 0
results['teamBRecentPoints'] = 0
results = results[['date', 'teamA', 'scoredA', 'teamB', 'scoredB', 'teamAPosition', 'teamBPosition', 'teamARecentScored', 'teamBRecentScored', 'teamARecentConceded', 'teamBRecentConceded', 'teamARecentPoints', 'teamBRecentPoints']]

# Display
#results

In [8]:
class LeagueTable:
    def __init__(self, teams):
        self.teams = list(teams)
        self.table = pd.DataFrame(np.array([teams, [0] * len(self.teams), [0] * len(self.teams), [0] * len(self.teams), [0] * len(self.teams), [0] * len(self.teams), [0] * len(self.teams), [0] * len(self.teams), [0] * len(self.teams)]).T, columns=['Team', 'P', 'W', 'D', 'L', 'F', 'A', 'GD', 'Pts'])
        self.positions = np.array(range(1, len(self.teams) + 1))
        self.sort_table()
        
    def sort_table(self):
        self.table.sort_values(by=['Team'], ascending=True, inplace=True, ignore_index=True)
        self.table.sort_values(by=['Pts', 'GD', 'F'], ascending=False, inplace=True, ignore_index=True)
        self.table.set_index(self.positions, inplace=True)
    
    def show_table(self):
        return self.table.head(len(self.teams))
    
    def add_result(self, team_a, scored_a, team_b, scored_b):
        # Team A
        self.table.loc[self.table['Team'] == team_a, 'P'] += 1
        self.table.loc[self.table['Team'] == team_a, 'W'] += 1 if int(scored_a) > int(scored_b) else 0
        self.table.loc[self.table['Team'] == team_a, 'D'] += 1 if int(scored_a) == int(scored_b) else 0
        self.table.loc[self.table['Team'] == team_a, 'L'] += 1 if int(scored_a) < int(scored_b) else 0
        self.table.loc[self.table['Team'] == team_a, 'F'] += int(scored_a)
        self.table.loc[self.table['Team'] == team_a, 'A'] += int(scored_b)
        self.table.loc[self.table['Team'] == team_a, 'GD'] += int(scored_a) - int(scored_b)
        self.table.loc[self.table['Team'] == team_a, 'Pts'] += 3 if int(scored_a) > int(scored_b) else 1 if int(scored_a) == int(scored_b) else 0
        # Team B
        self.table.loc[self.table['Team'] == team_b, 'P'] += 1
        self.table.loc[self.table['Team'] == team_b, 'W'] += 1 if int(scored_b) > int(scored_a) else 0
        self.table.loc[self.table['Team'] == team_b, 'D'] += 1 if int(scored_b) == int(scored_a) else 0
        self.table.loc[self.table['Team'] == team_b, 'L'] += 1 if int(scored_b) < int(scored_a) else 0
        self.table.loc[self.table['Team'] == team_b, 'F'] += int(scored_b)
        self.table.loc[self.table['Team'] == team_b, 'A'] += int(scored_a)
        self.table.loc[self.table['Team'] == team_b, 'GD'] += int(scored_b) - int(scored_a)
        self.table.loc[self.table['Team'] == team_b, 'Pts'] += 3 if int(scored_b) > int(scored_a) else 1 if int(scored_b) == int(scored_a) else 0
        # Reorder table
        self.sort_table()

In [9]:
table = LeagueTable(results['teamA'].unique())
table.show_table()

,Team,P,W,D,L,F,A,GD,Pts
1,Arsenal,0,0,0,0,0,0,0,0
2,Aston Villa,0,0,0,0,0,0,0,0
3,Bournemouth,0,0,0,0,0,0,0,0
4,Brighton,0,0,0,0,0,0,0,0
5,Burnley,0,0,0,0,0,0,0,0
6,Chelsea,0,0,0,0,0,0,0,0
7,Crystal Palace,0,0,0,0,0,0,0,0
8,Everton,0,0,0,0,0,0,0,0
9,Leicester,0,0,0,0,0,0,0,0
10,Liverpool,0,0,0,0,0,0,0,0


In [10]:
for index, row in results.iterrows():
    # Update features
    previous_games = game_data[:index*2]
    results.iloc[index, 5] = table.show_table()[table.show_table()['Team'] == row['teamA']].index[0].astype('uint16')
    results.iloc[index, 6] = table.show_table()[table.show_table()['Team'] == row['teamB']].index[0].astype('uint16')
    results.iloc[index, 7] = previous_games[previous_games['teamId'] == row['teamA']][-5:]['scored'].astype('uint16').sum()
    results.iloc[index, 8] = previous_games[previous_games['teamId'] == row['teamB']][-5:]['scored'].astype('uint16').sum()
    results.iloc[index, 9] = previous_games[previous_games['teamId'] == row['teamA']][-5:]['missed'].astype('uint16').sum()
    results.iloc[index, 10] = previous_games[previous_games['teamId'] == row['teamB']][-5:]['missed'].astype('uint16').sum()
    results.iloc[index, 11] = 3*previous_games[previous_games['teamId'] == row['teamA']][-5:]['wins'].astype('uint16').sum() + previous_games[previous_games['teamId'] == row['teamA']][-5:]['draws'].astype('uint16').sum()
    results.iloc[index, 12] = 3*previous_games[previous_games['teamId'] == row['teamB']][-5:]['wins'].astype('uint16').sum() + previous_games[previous_games['teamId'] == row['teamB']][-5:]['draws'].astype('uint16').sum()
    # Add result to table
    table.add_result(row['teamA'], row['scoredA'], row['teamB'], row['scoredB'])

In [11]:
happened_games = [('Aston Villa', 'Sheffield United', 0, 0), ('Man City', 'Arsenal', 3, 0),
                  ('Aston Villa', 'Chelsea', 1, 2), ('Bournemouth', 'Crystal Palace', 0, 2), ('Brighton', 'Arsenal', 2, 1), ('Everton', 'Liverpool', 0, 0), ('Man City', 'Burnley', 5, 0), ('Newcastle United', 'Sheffield United', 3, 0), ('Norwich', 'Southampton', 0, 3), ('Tottenham', 'Man Utd', 1, 1), ('Watford', 'Leicester', 1, 1), ('West Ham', 'Wolves', 0, 2),
                  ('Burnley', 'Watford', 1, 0), ('Chelsea', 'Man City', 2, 1), ('Leicester', 'Brighton', 0, 0), ('Liverpool', 'Crystal Palace', 4, 0), ('Man Utd', 'Sheffield United', 3, 0), ('Newcastle United', 'Aston Villa', 1, 1), ('Norwich', 'Everton', 0, 1), ('Southampton', 'Arsenal', 0, 2), ('Tottenham', 'West Ham', 2, 0), ('Wolves', 'Bournemouth', 1, 0),
                  ('Aston Villa', 'Wolves', 0, 1), ('Watford', 'Southampton', 1, 3), ('Crystal Palace', 'Burnley', 0, 1), ('Brighton', 'Man Utd', 0, 3), ('Arsenal', 'Norwich', 4, 0), ('Bournemouth', 'Newcastle United', 1, 4), ('Everton', 'Leicester', 2, 1), ('West Ham', 'Chelsea', 3, 2), ('Man City', 'Liverpool', 4, 0), ('Sheffield United', 'Tottenham', 3, 1),
                  ('Burnley', 'Sheffield United', 1, 1), ('Chelsea', 'Watford', 3, 0), ('Leicester', 'Crystal Palace', 3, 0), ('Liverpool', 'Aston Villa', 2, 0), ('Man Utd', 'Bournemouth', 5, 2), ('Newcastle United', 'West Ham', 2, 2), ('Norwich', 'Brighton', 0, 1), ('Southampton', 'Man City', 1, 0), ('Tottenham', 'Everton', 1, 0), ('Wolves', 'Arsenal', 0, 2),
                  ('Arsenal', 'Leicester', 1, 1), ('Aston Villa', 'Man Utd', 0, 3), ('Bournemouth', 'Tottenham', 0, 0), ('Brighton', 'Liverpool', 1, 3), ('Crystal Palace', 'Chelsea', 2, 3), ('Everton', 'Southampton', 1, 1), ('Man City', 'Newcastle United', 5, 0), ('Sheffield United', 'Wolves', 1, 0), ('Watford', 'Norwich', 2, 1), ('West Ham', 'Burnley', 0, 1),
                  ('Aston Villa', 'Crystal Palace', 2, 0), ('Bournemouth', 'Leicester', 4, 1), ('Brighton', 'Man City', 0, 5), ('Liverpool', 'Burnley', 1, 1), ('Man Utd', 'Southampton', 2, 2), ('Norwich', 'West Ham', 0, 4), ('Sheffield United', 'Chelsea', 3, 0), ('Tottenham', 'Arsenal', 2, 1), ('Watford', 'Newcastle United', 2, 1), ('Wolves', 'Everton', 3, 0),
                  ('Arsenal', 'Liverpool', 2, 1), ('Burnley', 'Wolves', 1, 1), ('Chelsea', 'Norwich', 1, 0), ('Crystal Palace', 'Man Utd', 0, 2), ('Everton', 'Aston Villa', 1, 1), ('Leicester', 'Sheffield United', 2, 0), ('Man City', 'Bournemouth', 2, 1), ('Newcastle United', 'Tottenham', 1, 3), ('Southampton', 'Brighton', 1, 1), ('West Ham', 'Watford', 3, 1)]
for game in happened_games:
    # Get win/draw/loss
    home_win = 1 if round(game[2]) > round(game[3]) else 0
    home_draw = 1 if round(game[2]) == round(game[3]) else 0
    home_loss = 1 if round(game[2]) < round(game[3]) else 0
    
    table.add_result(game[0], game[2], game[1], game[3])
    new_row_a = pd.DataFrame([['', '', '', '', '', '', '', '', game[2], game[3], '', '', '', home_win, home_draw, home_loss, '', '', game[0], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']], columns=game_data.columns)
    new_row_b = pd.DataFrame([['', '', '', '', '', '', '', '', game[3], game[2], '', '', '', home_loss, home_draw, home_win, '', '', game[1], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']], columns=game_data.columns)
    game_data = pd.concat([game_data, new_row_a, new_row_b], ignore_index=True)

In [12]:
table.show_table()

,Team,P,W,D,L,F,A,GD,Pts
1,Liverpool,36,30,3,3,77,29,48,93
2,Man City,36,24,3,9,93,35,58,75
3,Chelsea,36,19,6,11,64,49,15,63
4,Leicester,36,18,8,10,67,36,31,62
5,Man Utd,36,17,11,8,63,35,28,62
6,Wolves,36,14,14,8,49,38,11,56
7,Tottenham,36,15,10,11,57,46,11,55
8,Sheffield United,36,14,12,10,38,35,3,54
9,Arsenal,36,13,14,9,53,45,8,53
10,Burnley,36,14,9,13,40,48,-8,51


In [13]:
games_left = [('Norwich', 'Burnley'), ('Bournemouth', 'Southampton'), ('Tottenham', 'Leicester'), ('Brighton', 'Newcastle United'), ('Sheffield United', 'Everton'), ('Wolves', 'Crystal Palace'), ('Watford', 'Man City'), ('Aston Villa', 'Arsenal'), ('Man Utd', 'West Ham'), ('Liverpool', 'Chelsea'),
              ('Arsenal', 'Watford'), ('Burnley', 'Brighton'), ('Chelsea', 'Wolves'), ('Crystal Palace', 'Tottenham'), ('Everton', 'Bournemouth'), ('Leicester', 'Man Utd'), ('Man City', 'Norwich'), ('Newcastle United', 'Liverpool'), ('Southampton', 'Sheffield United'), ('West Ham', 'Aston Villa')]


games_left = pd.DataFrame(games_left, columns=['teamA', 'teamB'])
games_left.sample(5, random_state=RANDOM_STATE)

,teamA,teamB
18,Southampton,Sheffield United
1,Bournemouth,Southampton
19,West Ham,Aston Villa
8,Man Utd,West Ham
10,Arsenal,Watford


In [14]:
X_train = results[['teamA', 'teamB', 'teamAPosition', 'teamBPosition', 'teamARecentScored', 'teamBRecentScored', 'teamARecentConceded', 'teamBRecentConceded', 'teamARecentPoints', 'teamBRecentPoints']]
X_train.fillna(0, inplace=True)
Y_train = results[['scoredA', 'scoredB']]

/var/folders/96/qqlkbqys6kqb27dyfw97gdz40000gp/T/ipykernel_48825/2975102970.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(0, inplace=True)


In [15]:
# Returns 1 if the team won the Premier League in the previous five seasons
def is_previous_winner(team):
    winners = ['Man City', 'Chelsea', 'Leicester']
    return 1 if team in winners else 0

# Returns 1 if the team finished in the top 4 of the Premier League in the previous five seasons
def is_previous_top_4(team):
    top_4 = ['Man City', 'Liverpool', 'Chelsea', 'Tottenham', 'Man Utd', 'Leicester', 'Arsenal']
    return 1 if team in top_4 else 0

# Returns 1 if the team played at least one season in the Championship (second division) in the previous five seasons
def is_championship(team):
    championships = ['Bournemouth', 'Watford', 'Norwich', 'Burnley', 'Newcastle United', 'Brighton', 'Wolves', 'Sheffield United', 'Aston Villa']
    return 1 if team in championships else 0

X_train['teamAPrevWinner'] = X_train['teamA'].apply(lambda x: is_previous_winner(x))
X_train['teamBPrevWinner'] = X_train['teamB'].apply(lambda x: is_previous_winner(x))
X_train['teamAPrevTop4'] = X_train['teamA'].apply(lambda x: is_previous_top_4(x))
X_train['teamBPrevTop4'] = X_train['teamB'].apply(lambda x: is_previous_top_4(x))
X_train['teamAPrevChampionship'] = X_train['teamA'].apply(lambda x: is_championship(x))
X_train['teamBPrevChampionship'] = X_train['teamB'].apply(lambda x: is_championship(x))

X_train.drop(['teamA', 'teamB'], inplace=True, axis=1)

/var/folders/96/qqlkbqys6kqb27dyfw97gdz40000gp/T/ipykernel_48825/2383648321.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['teamAPrevWinner'] = X_train['teamA'].apply(lambda x: is_previous_winner(x))
/var/folders/96/qqlkbqys6kqb27dyfw97gdz40000gp/T/ipykernel_48825/2383648321.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['teamBPrevWinner'] = X_train['teamB'].apply(lambda x: is_previous_winner(x))


In [16]:
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_train[:,0:2] *= 2.5
X_train[:,2:8] *= 1.5

In [17]:
model = MultiOutputRegressor(KNeighborsRegressor(n_neighbors=3, weights='distance')).fit(X_train, Y_train)

In [18]:
for index, row in games_left.iterrows():
    # Get recent form features
    a_position = table.show_table()[table.show_table()['Team'] == row['teamA']].index[0].astype('uint16')
    b_position = table.show_table()[table.show_table()['Team'] == row['teamB']].index[0].astype('uint16')
    a_scored = game_data[game_data['teamId'] == row['teamA']][-5:]['scored'].astype('uint16').sum()
    b_scored = game_data[game_data['teamId'] == row['teamB']][-5:]['scored'].astype('uint16').sum()
    a_conceded = game_data[game_data['teamId'] == row['teamA']][-5:]['missed'].astype('uint16').sum()
    b_conceded = game_data[game_data['teamId'] == row['teamB']][-5:]['missed'].astype('uint16').sum()
    a_points = 3*game_data[game_data['teamId'] == row['teamA']][-5:]['wins'].astype('uint16').sum() + game_data[game_data['teamId'] == row['teamA']][-5:]['draws'].astype('uint16').sum()
    b_points = 3*game_data[game_data['teamId'] == row['teamB']][-5:]['wins'].astype('uint16').sum() + game_data[game_data['teamId'] == row['teamB']][-5:]['draws'].astype('uint16').sum()
    
    
    # Get extra features
    a_prev = is_previous_winner(row['teamA'])
    b_prev = is_previous_winner(row['teamB'])
    a_top4 = is_previous_top_4(row['teamA'])
    b_top4 = is_previous_top_4(row['teamB'])
    a_cham = is_championship(row['teamA'])
    b_cham = is_championship(row['teamB'])
    
    # Make game prediction
    X_pred = np.array([a_position, b_position, a_scored, b_scored, a_conceded, b_conceded, a_points, b_points, a_prev, b_prev, a_top4, b_top4, a_cham, b_cham]).reshape(1, -1)
    X_pred = scaler.transform(X_pred)
    X_pred[:,0:2] *= 2.5
    X_pred[:,2:8] *= 1.5
    goals = model.predict(X_pred)
    
    # Add result to the table
    table.add_result(row['teamA'], round(goals[0][0]), row['teamB'], round(goals[0][1]))
    
    # Get win/draw/loss
    home_win = 1 if round(goals[0][0]) > round(goals[0][1]) else 0
    home_draw = 1 if round(goals[0][0]) == round(goals[0][1]) else 0
    home_loss = 1 if round(goals[0][0]) < round(goals[0][1]) else 0
    
    # Save the result (updating recent form)
    new_row_a = pd.DataFrame([['', '', '', '', '', '', '', '', goals[0][0], goals[0][1], '', '', '', home_win, home_draw, home_loss, '', '', row['teamA'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']], columns=game_data.columns)
    new_row_b = pd.DataFrame([['', '', '', '', '', '', '', '', goals[0][1], goals[0][0], '', '', '', home_loss, home_draw, home_win, '', '', row['teamB'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']], columns=game_data.columns)
    game_data = pd.concat([game_data, new_row_a, new_row_b], ignore_index=True)

# Show the final table
table.show_table()

/Users/gowthamramakrishnan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/gowthamramakrishnan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/gowthamramakrishnan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/gowthamramakrishnan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/Users/gowthamramakrishnan/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with f

,Team,P,W,D,L,F,A,GD,Pts
1,Liverpool,38,31,3,4,80,32,48,96
2,Man City,38,25,4,9,97,38,59,79
3,Man Utd,38,19,11,8,67,36,31,68
4,Chelsea,38,20,7,11,67,51,16,67
5,Leicester,38,19,8,11,70,39,31,65
6,Arsenal,38,15,14,9,58,47,11,59
7,Wolves,38,14,16,8,51,40,11,58
8,Burnley,38,16,9,13,43,48,-5,57
9,Tottenham,38,15,11,12,59,49,10,56
10,Sheffield United,38,14,14,10,40,37,3,56
